In [1]:
import pandas as pd
import math as math
import numpy as np
import sympy as sy
from sympy import series, Symbol
from sympy.functions import sin, cos, exp
from sympy.plotting import plot
from sympy import lambdify
from numpy import linspace
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
### loads the MNIST training data and plots the first 30 images.
# Read MNIST csv file into dataframe
df = pd.read_csv('mnist_train.csv')
#df2 = pd.DataFrame(-255*np.ones(len(df.pixel1)),columns=list('a'))

# to insert -1 prepend
df.insert(1, "a", -1) 
# Merge pixels into feature column and keep only feature
#axis 1 apply function to each row
df ['feature'] = df.apply(lambda row: row.values [1 :], axis =1)
df = df[['feature','label']]
df

,feature,label
0,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
1,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
2,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
3,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4
4,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
...,...,...
41995,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
41996,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
41997,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",7
41998,"[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",6


In [6]:
#First, extract the indices of all the 0’s 
#and randomly separate the samples into equal-sized training and testing groups.
zero=df.loc[lambda df: df['label'] == 0, :]
train0, test0 =train_test_split(zero, train_size = 0.5)
#msk = np.random.rand(len(zero)) < 0.5
#train0 = zero[msk]
#test0 = zero[~msk]
#train0
train0_label=train0.label
train0_feature=train0.feature
test0_label=test0.label
test0_feature=test0.feature

##do the same for the 1’s.
one=df.loc[lambda df: df['label'] == 1, :]
train1, test1 =train_test_split(one, train_size = 0.5)
#msk1 = np.random.rand(len(one)) < 0.5
#train1 = one[msk1]
#test1 = one[~msk1]
train1_label=train1.label
train1_feature=train1.feature
test1_label=test1.label
test1_feature=test1.feature

# group test 0,1 together and train 0,1 together
test_feature =np.hstack([np.array(test0_feature), np.array(test1_feature)])/255 
train_feature =np.hstack([train0_feature, train1_feature])/255
test_label =np.hstack([np.array(test0_label),np.array(test1_label)]) 
train_label =np.hstack([train0_label, train1_label])

In [ ]:
##gradient method
A=np.vstack(train_feature)
Y=np.vstack(train_label)
#soln=np.linalg.lstsq(A, Y,rcond=None)[0]
CER_M = 1074*1.2 # Set the max number of wrong output is 1.2 times of the original one

cur_B=np.array([np.ones(785)])
max_t = 1000000
t=0
N=785
r=0.3  
Y = np.array(train_label)
l=len(A)
X=[]
def LSclassifier(x):
    if x < 1:
        return -1
    else:
        return 1
    return x
def Labelclass(x):
    if x ==0:
        return -1
    else:
        return 1
    return x
# set the initial condition
for i in range(0,l,1):
    p=np.dot(np.array(train_feature[i]),np.array(cur_B.T))
    X.append(p)
TrainOutput=[LSclassifier(x) for x in X]           
TrainClass=[Labelclass(x) for x in Y]
CER=np.count_nonzero(np.array(TrainClass)-np.array(TrainOutput))

while CER>=2066:
    xX=[]
    for i in range(0,l,1):
        cur_B=np.random.rand(1,785)
        p=np.dot(np.array(train_feature[i]),np.array(cur_B.T))
        xX.append(p)
    TrainOutput=[LSclassifier(x) for x in xX]           
    CER=np.count_nonzero(np.array(TrainClass)-np.array(TrainOutput))
else:
        CER=CER
        cur_B=cur_B

In [ ]:
#find the solution
while (CER > CER_M):
        pre_B = cur_B
        cur_B = cur_B+((Y[i]-A[i]*cur_B)*A[i])
        t=t+1
        X=[]
        for i in range(0,l,1):
            p=np.dot(np.array(train_feature[i]),np.array(cur_B.T))
            X.append(p)
        TrainOutput=[LSclassifier(x) for x in X]
        CER=np.count_nonzero(np.array(TrainClass)-np.array(TrainOutput))
        print("Iteration",t,"MSE value is",CER)
else:
    print("Final Iteration",t,"MSE value is",CER)
    #"The local minimum occurs at", cur_B,